# Introduction

A decorator is a function that takes another function and extends the behaviour of the latter function without explicitly modifying it.

Provide a simple syntax for calling higher order functions, which do either
- take one or more functions as arguments
- return a function as a result

Question:
- How does a decorator extend the behaviour of a function without modifying it?

### Functions


Pure functions return a value based on the given argument. A decorator is a pure function.

#### First-Class Objects


Functions can be passed around and used as arguments.

In [3]:
# 'greet_bob'is parameterised with a function. It chooses what argument to invoke the passed functions with.

def say_hello(name):
    return f"Hello {name}"

def be_awesome(name):
    return f"Yo {name}, together we are the awesomest!"

def greet_bob(greeter_func):
    return greeter_func("Bob")

In [6]:
# A reference to the function 'say_hello' is passed to 'greet_bob'. It is called inside the function definition of greet_bob.

greet_bob(say_hello)

'Hello Bob'

In [7]:
greet_bob(be_awesome)

'Yo Bob, together we are the awesomest!'

#### Inner Functions


In [10]:
def parent():
    print("Printing from the parent function.")

    def first_child():
        print("Printing from first child() function.")
    
    def second_child():
        print("Printing from the second_child() function.")

    first_child()
    second_child()
    

#### Returning Functions from Functions

In [16]:
# The function is being returned without parentheses; therefore, a reference to the function is returned instead of evaluating the function.parent

def parent(num):
    def first_child():
        return "Hi, I am Emma"
    
    def second_child():
        return "Call me Liam"

    if num==1:
        return first_child
    else:
        return second_child

In [18]:
parent(1)

<function __main__.parent.<locals>.first_child()>

In [20]:
first = parent(1)
second = parent(2)

In [21]:
first

<function __main__.parent.<locals>.first_child()>

In [22]:
second

<function __main__.parent.<locals>.second_child()>

Even though first and second refer to the local functions, first_child() and second_child(), we have references to them and they can be invoked. This is because parent() returns references to the functions.

In [25]:
first()

'Hi, I am Emma'

In [26]:
second()

'Call me Liam'

### Simple Decorators



In [31]:
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

def say_whee():
    print("Whee!")

say_whee = my_decorator(say_whee)
say_whee

<function __main__.my_decorator.<locals>.wrapper()>

wrapper() has a reference to the original function say_whee() and will call say_whee() between two calls to print when wrapper() is called.

Decorators wrap functions, modifying their behaviour.

In [24]:
from datetime import datetime

def not_during_the_night(func):
    def wrapper():
        if 7 <= datetime.now().hour < 22:
            func()
        else:
            pass # Hush, the neighbours are sleeping.
    return wrapper

def say_whee():
    print("Whee!")

say_whee = not_during_the_night(say_whee)
say_whee()

Whee!


 #### Syntactic Sugar!


In [21]:
def my_decorator(func):
    def wrapper():
        print("something before")
        func()
        print("something after")
    return wrapper


@my_decorator
def say_whee():
    print("whee!")    

@my_decorator is just an easier way of saying

say_whee = my_decorator(say_whee)

 #### Reusing Decorators


A decorator is just a regular Python function. All the usual tools for easy reusability are available. For example, a decorator can be in its own module and imported.

In [3]:
from decorators import do_twice

@do_twice
def say_whee():
    print("Whee!")

In [4]:
say_whee

<function __main__.say_whee()>

 #### Decorating Functions With Arguments
 

In [1]:
from decorators import do_twice

@do_twice
def greet(name):
    print(f"Hello {name}")

In [29]:
greet('World')

Hello World
Hello World


(None, None)

#### Returning Values From Decorated Functions


wrapper_do_twice(return_greeting(name))()



In [30]:
@do_twice
def return_greeting(name):
    print("Creating greeting")
    return f"Hi {name}"

hi_adam = return_greeting("Adam")
print(hi_adam)

Creating greeting
Creating greeting
('Hi Adam', 'Hi Adam')


 #### Who Are You, Really?

Introspection is the ability of an object to know about its own attributes at runtime.

In [32]:
print

<function print>

In [33]:
print.__name__

'print'

In [34]:
help(print)

Help on built-in function print in module builtins:

print(...)
    print(value, ..., sep=' ', end='\n', file=sys.stdout, flush=False)
    
    Prints the values to a stream, or to sys.stdout by default.
    Optional keyword arguments:
    file:  a file-like object (stream); defaults to the current sys.stdout.
    sep:   string inserted between values, default a space.
    end:   string appended after the last value, default a newline.
    flush: whether to forcibly flush the stream.



In [6]:
say_whee

<function __main__.say_whee()>

In [5]:
say_whee.__name__

'say_whee'

In [7]:
help(say_whee)

Help on function say_whee in module __main__:

say_whee()



### A Few Real World Examples


Good boilerplate for a decorator.

In [ ]:
import functools

def decorator(func):
    @functools.wraps(func)
    def wrapper_decorator(*args, **kwargs):
        # Do something before
        value = func(*args, **kwargs)
        # Do something after
        return value
    return wrapper_decorator

 #### Timing Functions


In [36]:
import functools
import time

def timer(func):
    """Print the runtime of the decorated function"""
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = time.perf_counter()    #  1
        value = func(*args, **kwargs)
        end_time = time.perf_counter()
        run_time = end_time - start_time
        print(f"Finished {func.__name__!r} in {run_time:.4f} secs")
        return value
    return wrapper_timer

@timer
def waste_some_time(num_times):
    for _ in range(num_times):
        sum([i**2 for i in range(10_000)])
    return 'finished'

In [35]:
waste_some_time(999)

Finished 'waste_some_time' in -4.9293 secs


'finished'

In [43]:
import timeit

s = """\
for _ in range(10):
    sum([i**2 for i in range(10_000)])
"""

timeit.timeit(stmt=s, number=100)

5.8546960999956354

 #### Debugging Code


In [5]:
from decorators import debug

@debug
def make_greeting(name, age=None):
    if age is None:
        return f"Howdy {name}!"
    else: 
        return f"Whoa {name}! {age} already, you are growing up!"

In [6]:
make_greeting("Benjamin")

Calling make_greeting('Benjamin', ||)
'make_greeting' returned 'Howdy Benjamin!'


'Howdy Benjamin!'

In [1]:
import math
from decorators import debug

# Apply a decorator to a standard library function
math.factorial = debug(math.factorial)

def approximate_e(terms = 18):
    return sum(1/math.factorial(n) for n in range(terms))

In [2]:
approximate_e(5)

Calling factorial(0)
'factorial' returned 1
Calling factorial(1)
'factorial' returned 1
Calling factorial(2)
'factorial' returned 2
Calling factorial(3)
'factorial' returned 6
Calling factorial(4)
'factorial' returned 24


2.708333333333333

 #### Slowing Down Code


In [5]:
from decorators import slow_down


@slow_down
def countdown(from_number):
    if from_number <1:
        print("Liftoff!")
    else:
        print(from_number)
        countdown(from_number - 1)

In [6]:
countdown(3)

3
2
1
Liftoff!
-
-
-
-


 #### Registering Plugins


In [6]:
import random
PLUGINS = dict()

def register(func):
    """Register a function as a plug-in"""
    PLUGINS[func.__name__] = func
    return func

@register
def say_hello(name):
    return f"Hello {name}"

@register
def be_awesome(name):
    return f"Yo {name}, together we are the awesomest!"

In [7]:
def randomly_greet(name):
    greeter, greeter_func = random.choice(list(PLUGINS.items()))
    print(f"Using {greeter!r}")
    return greeter_func(name)

In [4]:
PLUGINS

{'say_hello': <function __main__.say_hello(name)>,
 'be_awesome': <function __main__.be_awesome(name)>}

In [11]:
randomly_greet('Alice')

Using 'say_hello'


'Hello Alice'

Testing that registers call a function when it is defined

In [1]:
from decorators import debug
from decorators import printer

@printer
def test():
    return 'Test'

In [2]:
test()

hello


 #### Is the User Logged In?


In [ ]:
from flask import Flask, g, request, redirect, url_for
import functools



### Fancy Decorators
 

#### Decorating Classes

There are two ways to use decorators on classes:
- decorate the methods of the class 
- decorate the whole class

@classmethod, @staticmethod and @property are examples of decorators used to define methods inside a class namespace that are not connected to a particular instance of that class.

In [13]:
#The definition of this Circle class uses the @classmethod, @staticmethod and @property built-in decorators.staticmethod

class Circle:
    def __init__(self, radius):
        self._radius = radius

    @property
    def radius(self):        # A mutable property because of the setter.
        """Get value of radius"""    
        return self._radius
    
    @radius.setter
    def radius(self, value):
        if value >= 0:
            self._radius = value
        else:
            raise ValueError("Radius must be positive")

    @property
    def area(self):           # An immutable property because of no setter.
        """Calculate the area inside circle"""
        return self.pi() * self.radius**2

    def cylinder_volume(self, height):
        """Calculate volume of cylinder with circle as base"""
        return self.area * height

    @classmethod
    def unit_circle(cls): # depends on class. a factory method.
        """Factory method creating a circle with radius 1"""
        return cls(1)

    @staticmethod
    def pi():     # uses namespace but doesn't depend on class or instances.
        """Value of pi, could use math.pi instead though"""
        return 3.1415926535

In [16]:
c = Circle(5)
c.radius

5

In [17]:
c.area

78.5398163375

In [19]:
c.radius = 2
c.area

12.566370614

In [20]:
c.area = 100

AttributeError: can't set attribute 'area'

In [21]:
c.cylinder_volume(height=4)

50.265482456

In [22]:
c.radius = -1

ValueError: Radius must be positive

In [23]:
c = Circle.unit_circle()
c.radius

1

In [25]:
Circle.pi()

3.1415926535

In [2]:
# Let's define a class where we decorate some of its methods using the @debug and @timer decorators from earlier.

from decorators import debug, timer

class TimeWaster:
    @debug
    def __init__(self, max_num):
        self.max_num = max_num
    
    @timer
    def waste_time(self, num_times):
        for _ in range(num_times):
            sum([i**2 for i in range(self.max_num)])


In [7]:
tw = TimeWaster(1000)

tw.waste_time(999)


Calling __init__(<__main__.TimeWaster object at 0x0000022D1B7B7100>, 1000)
'__init__' returned None
Finished 'waste_time' in 0.5813 secs


Decorating the whole class

- a common use case is to change the definition of a class dynamically; a simpler alternative to metaclasses

In [8]:
from dataclasses import dataclass

@dataclass
class PlayingCard:
    rank: str
    suit: str

# PlayingCard = dataclass(PlayingCard)

In [9]:
from decorators import time

@timer
class TimeWaster:
    def __init__(self, max_num) -> None:
        self.max_num = max_num

    def waste_time(self, num_times):
        for _ in range(num_times):
            sum([i**2 for i in range(num_times)])

In [10]:
tw = TimeWaster(1000)

tw.waste_time(999)

Finished 'TimeWaster' in 0.0000 secs



 #### Nesting Decorators


In [13]:
from decorators import debug, do_twice

@do_twice
@debug
def greet(name):
    print(f"Hello {name}")

In [14]:
greet("eva")

Calling greet('eva')
Hello eva
'greet' returned None
Calling greet('eva')
Hello eva
'greet' returned None


 #### Decorators With Arguments


In [5]:
from decorators import repeat

@repeat
def greet(name, num_times):
    print(f"Hello {name}")

In [22]:
import functools

def repeat(num_times):
    def decorator_repeat(func):
        @functools.wraps(func)
        def wrapper_repeat(*args, **kwargs):
            for _ in range(num_times):
                value = func(*args, **kwargs)
            return value
        return wrapper_repeat
    return decorator_repeat


In [23]:
@repeat(num_times = 4)
def greet():
    print('hi')

In [26]:
greet()

hi
hi
hi
hi


In [34]:
def argument(value):
    def decorator_argument(func):
        @functools.wraps(func)
        def wrapper_argument(*args, **kwargs):
            print(value)
            return func(*args, **kwargs)
        return wrapper_argument
    return decorator_argument

In [36]:
@argument(10)
def practice():
    return


In [37]:
practice

<function __main__.practice()>

 #### Both Please, But Never Mind the Bread


In [ ]:
def name(_func=None, *, kw1=val1, kw2=val2):           # 1
    def decorator_name(func):
        ... # Create and return a wrapper function.
    
    if _func is None:
        return decorator_name                           # 2
    else:
        return decorator_name(_func)

1. If *name* has been called without arguments, the decorated function will

In [89]:
from decorators import debug

def tester(func):
    def wrapper_tester():
        print(func)
        return func()
    return wrapper_tester

In [90]:
@tester
def tester_companion():
    return 5

In [92]:
tester_companion()

# tester(tester_companion)
    # func = tester_companion
# wrapper_tester(tester_companion)


<function tester_companion at 0x000001A5265237F0>


5

In [131]:
def repeat(_func=None, *, num_times=2):
    def decorator_repeat(func):
        def wrapper_repeat(*args, **kwargs):
            for _ in range(num_times):
                value = func(*args, **kwargs)
            return value
        return wrapper_repeat

    if _func is None:
        return decorator_repeat
    else:
        return decorator_repeat(_func)

In [155]:
@repeat
def whee():
    print("Whee!")



In [156]:
whee()

Whee!
Whee!


 #### Stateful Decorators

In [1]:
from decorators import count_calls


@count_calls
def say_whee():
    print("Whee!")

In [6]:
say_whee()
say_whee.num_calls

Call 4 of 'say_whee'
Whee!


4

 #### Classes as Decorators


In [9]:
class Counter:
    def __init__(self, start=0):
        self.count = start

    def __call__(self):
        self.count += 1
        print(f"Current count is {self.count}")

In [13]:
counter = Counter()
counter()

Current count is 1


In [15]:

counter.count

2

In [27]:
import functools

class CountCalls:
    def __init__(self, func):
        functools.update_wrapper(self, func)
        self.func = func
        self.num_calls = 0

    def __call__(self, *args, **kwargs):
        self.num_calls += 1
        print(f"Call {self.num_calls} of {self.func.__name__!r}")
        return self.func(*args, **kwargs)



In [28]:
@CountCalls
def say_whee():
    print("Wheee!")

In [29]:
say_whee

### More Real World Examples


 #### Slowing Down Code, Revisited

In [30]:
import functools
import time

def slow_down(_func = None, *, rate=1):
    def decorator_slow_down(func):
        @functools.wraps(func)
        def wrapper_slow_down(*args, **kwargs):
            time.sleep(rate)
            return func(*args, **kwargs)
        return wrapper_slow_down
    if _func is None:
        return decorator_slow_down
    else:
        return decorator_slow_down(_func)

In [35]:
@slow_down(rate=2)
def countdown(from_number):
    if from_number < 1:
        print("Liftoff!")
    else:
        print(from_number)
        countdown(from_number-1)

In [36]:
countdown(4)

4
3
2
1
Liftoff!



 #### Creating Singletons


In [51]:
import functools

def singleton(cls):
    """Make a class a Singleton class (only one instance)"""
    # @functools.wraps(cls)
    def wrapper_singleton(*args, **kwargs):
        if not wrapper_singleton.instance:
            wrapper_singleton.instance = cls(*args, **kwargs)
        return wrapper_singleton.instance
    wrapper_singleton.instance = None
    return wrapper_singleton    


In [52]:
@singleton
class TheOne:
    pass

In [53]:
first_one = TheOne()

In [54]:
first_one

 #### Caching Return Values


In [56]:
from decorators import count_calls

@count_calls
def fibonacci(num):
    if num < 2:
        return num
    return fibonacci(num-1) + fibonacci(num - 2)

In [61]:
fibonacci.num_calls

354

In [62]:
import functools
from decorators import count_calls

def cache(func):
    """Keep a cache of previous calls"""
    @functools.wraps(func)
    def wrapper_cache(*args, **kwargs):
        cache_key = args + tuple(kwargs.items())
        if cache_key not in wrapper_cache.cache:
            wrapper_cache.cache[cache_key] = func(*args, **kwargs)
        return wrapper_cache.cache[cache_key]
    wrapper_cache.cache = dict()
    return wrapper_cache

In [67]:
@cache
@count_calls
def fibonacci(num):
    if num < 2:
        return num
    return fibonacci(num - 1) + fibonacci(num - 2)

In [70]:
fibonacci(11)

Call 12 of 'fibonacci'


89

 #### Adding Information About Units


In [4]:
def set_unit(unit):
    """Register a unit on a function"""
    def decorator_set_unit(func):
        func.unit = unit
        return func
    return decorator_set_unit 

In [5]:
import math
@set_unit("cm^3")
def volume(radius, height):
    return math.pi * radius **2 * height

In [6]:
volume.unit

'cm^3'

In [8]:
import pint
ureg = pint.UnitRegistry()
vol = volume(3,5) * ureg(volume.unit)

In [10]:
vol

141.3716694115407 <Unit('centimeter ** 3')>

In [12]:
vol.to("cubic inches")

8.627028576414954 <Unit('inch ** 3')>

In [15]:
vol.to("gallons") #magnitude

0.03734644405374439 <Unit('gallon')>

In [18]:
import functools
def use_unit(unit):
    use_unit.ureg = pint.UnitRegistry()
    def decorator_use_unit(func):
        @functools.wraps(func)
        def wrapper_use_unit(*args, **kwargs):
            value = func(*args, **kwargs)
            return value * use_unit.ureg(unit)
        return wrapper_use_unit
    return decorator_use_unit

In [19]:
@use_unit("meters per second")
def average_speed(distance, duration):
    return distance / duration

In [20]:
bolt = average_speed(100, 9.58)

In [22]:
bolt

10.438413361169102 <Unit('meter / second')>

In [24]:
bolt.to("km per hour")

37.578288100208766 <Unit('kilometer / hour')>

In [25]:
bolt.to("mph").m

23.35006567906474

#### Validating JSON
### Conclusion
### Further Reading